# Multinomial Regression

In [1]:
import pandas as pd
import statsmodels.api as sm

## Loading Data

In this activity, we will work with the 'abalone' dataset, which is in this repo as a .csv file

In [2]:
data = pd.read_csv("abalone.csv")
data.head()

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


## Fitting a Multinomial Regression Model

We are going to fit a model to determine which characteristics of our abalone are significantly different between adult males (M), adult females (F), and infants (I)

The Sex column is therefore our dependent variable, which we will denote as `y`, and `X` will be our independent variables (everything else)

In [3]:
X = data[data.columns[~data.columns.isin(['Sex'])]]
y = data['Sex']

The snippet below fits our model. `sm.MNLogit` is a multinomial logistic regression (classifier with multiple independent variables), and we fit this on our dependent `y` and our independent `sm.add_constant(X)` (`X` plus a column of 1s to add an intercept)

In [4]:
mn = sm.MNLogit(y,sm.add_constant(X))

In [5]:
model = mn.fit()
print_model = model.summary()
print(print_model)

Optimization terminated successfully.
         Current function value: 0.854590
         Iterations 8
                          MNLogit Regression Results                          
Dep. Variable:                    Sex   No. Observations:                 4177
Model:                        MNLogit   Df Residuals:                     4159
Method:                           MLE   Df Model:                           16
Date:                Wed, 17 Apr 2024   Pseudo R-squ.:                  0.2204
Time:                        21:13:18   Log-Likelihood:                -3569.6
converged:                       True   LL-Null:                       -4578.9
Covariance Type:            nonrobust   LLR p-value:                     0.000
         Sex=I       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const              2.8410      0.505      5.627      0.000       1.851       3.831
Length           

The statsmodels module's output has K-1 equations (in this case two equations, since we have three classes), which show coefficients for each group as compared against a reference group. 

In this example, the reference group was chosen to be Female (it is alphabetical by default). The first set of results corresponds to Infant vs Female - we can see Sex = I in the top left. If a result is significant ($p< 0.05$) in this output, that means it is statisitically different between the two groups.
The second set of outputs corresponds to Sex = M, so it is looking for differences between adult Females and adult Males.

**Note**: As we have set it up, this model does not make direct comparison between Males and Infants. We could calculate this directly, but it would require a fair bit of work. Instead, if we are interested in making this comparison, we could re-run the model while specifying a different baseline. To do this, we manually set the order using `pandas`. If we run the snippet below before data splitting, Male would become the default, as it is the 'first' category. 

`data['Sex'] = pd.Categorical(data['Sex'], categories=['M', 'F', 'I'], ordered=True)`


## Accessing Model Parameters

In statsmodels, the `fit()` method returns a `Result` object. The model coefficients, standard errors, p-values, etc., are all available from this Result object.

These are conveniently stored as Pandas dataframes with the parameter name as the dataframe index. This allows us to programmitcally access the values in the summary above if we need them for downstream calculations.

In [15]:
params = model.params

# column names are not defined by default
params.columns = ["I vs F", "M vs F"]

params


,I vs F,M vs F
const,2.840995,2.521223
Length,17.681684,-1.015661
Diameter,-13.048868,-4.962977
Height,-8.107566,-3.176845
Whole_weight,-6.399546,-0.131395
Shucked_weight,5.246396,3.048409
Viscera_weight,-13.217943,-2.166477
Shell_weight,5.595137,0.474669
Rings,-0.196681,0.005943


Here are some of the relevant attribute values for a Logistic Regression.


|Attr/func|Description|
| ------------- |-------------|
|params|Estimated model parameters. Appears as coef when calling summary() on a fitted model|
|bse|Standard error|
|tvalues|A coefficient's t-statistic|
|pvalues|The model's p-value|
|conf_int(alpha)|Method that calculates the confidence interval for the estimated parameters. To call: model.conf_int(0.05)|



## Evaluating Multinomial Regression Model

So what do all these numbers mean, physically?

Two Ways to Assess:

1. Examine the model output for individual variables
2. Use .pred_table() method for overall model fit

### The Model Output Coefficients

The coefficients in the model output represent the log of ratios between two probabilities: the probability of belonging to a group of interest vs. the probability of belonging to the reference group. 

> Note: before making any physical interpretation about the effect of a variable, we should check that the p-value is less than our threshold $\alpha$ (i.e. 0.05)

For example, the coefficient for `Length` between Infants and Females is 17.7. This means for each unit increase in length, the sample is $e^{17.7}$ times more likely to belong to the class of interest. 

But wait! That makes no sense, why is a longer abalone so much more likely to be an infant? What's going on? 

Firstly, a unit increase in `length` is a BIG deal. Looking at the actual values in our `length` column, the mean is 0.52, with a standard deviation of 0.12. Suddenly increasing by 1.0 would be a huge deal - if we added that to the average abalone (0.52, now 1.52) it would be almost twice the length of the largest observed (0.82). So it makes sense that this would have a huge effect on the odds, especially if there is actually a big difference in length between Females and Infants.


In [7]:
data['Length'].describe()

count    4177.000000
mean        0.523992
std         0.120093
min         0.075000
25%         0.450000
50%         0.545000
75%         0.615000
max         0.815000
Name: Length, dtype: float64

That still doesn't explain the sign though - why is a longer abalone more likely to be infant? Aren't females bigger?

In [8]:
data.groupby('Sex')['Length'].mean()

Sex
F    0.579093
I    0.427746
M    0.561391
Name: Length, dtype: float64

They are! But the key thing to remember is that this coefficient assumes that the `length` value is increasing by 1.0 with **EVERYTHING ELSE THE SAME**.

Is this realistic? No! How is an abalone going to get much longer without increasing its other dimensions and weight? If we look at the coefficients related to other features like `viscera weight` and `diameter`, we see large negative values, indicating that increases in these values mean an example is much more likely to be Female.

Because so many of these features are dependent/ related to eachother, only when adding them up into the full model and allowing positives and negative coefficients to cancel do we get a full reasonable prediction of the `Sex`.

What do we do with the information given with this larger `length` coefficient then? We can interpret it that Infants are long and skinny *relative to their size*, so if an abalone is longer without being correspondingly wide and heavy, thats a sign that it might be an Infant.

Importantly, if we don't have access to the other information, and we were only predicting based on the size, we would get a different result. 

In [9]:
X = data['Length']
lr = sm.MNLogit(y,sm.add_constant(X))
small_model = lr.fit()
print_model = small_model.summary()
print(print_model)

Optimization terminated successfully.
         Current function value: 0.925998
         Iterations 6
                          MNLogit Regression Results                          
Dep. Variable:                    Sex   No. Observations:                 4177
Model:                        MNLogit   Df Residuals:                     4173
Method:                           MLE   Df Model:                            2
Date:                Wed, 17 Apr 2024   Pseudo R-squ.:                  0.1553
Time:                        21:13:18   Log-Likelihood:                -3867.9
converged:                       True   LL-Null:                       -4578.9
Covariance Type:            nonrobust   LLR p-value:                1.624e-309
     Sex=I       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.1382      0.264     26.997      0.000       6.620       7.656
Length       -13.9170      0.

In this version of the model that includes only `length`, a longer abalone is much less likely to be an `infant` - which makes sense, because our only indicator of overall size here is `length` - we don't include the confounding variables related to width and weight. Our model with many covariates allows us to capture a more complex view of the differences - like how infants seem overall to be small, but long and skinny. 

### The Prediction Table

The prediction table is a 'confusion matrix', which shows the distribution of the actual classes versus the predicted classes.

This allows us to empirically assess the model's predictive accuracy, as well as the types of errors it makes. In a perfect model, all examples would fall along the diagonal, indicating they have been classified correctly. 

The raw prediction table output looks like this:

In [10]:
pred_table = model.pred_table()
pred_table

array([[ 449.,  215.,  643.],
       [  69., 1108.,  165.],
       [ 385.,  351.,  792.]])

We can enhance this for clarity:

In [11]:
# check that this matches the category order
# alphabetical by default if you haven't changed it
categories = ['female', 'infant', 'male'] 

labeled_pred_table = pd.DataFrame(pred_table, index=categories, columns=categories)

labeled_pred_table

,female,infant,male
female,449.0,215.0,643.0
infant,69.0,1108.0,165.0
male,385.0,351.0,792.0


By default, the rows represent the actual classes, and the columns correspond to the predicted classes.

We can interpret row by row how the model is doing.

For females (first row), we can see the model is not doing a very good job, as we see more females are predicted as males than correctly as females, with an overall accuracy of $\frac{449}{449+215+643} = 0.34$ for females.

For infants (second row), the model is quite effective, as the large majority of infants are correctly identified, with an accuracy of $\frac{1108}{69+1108+165} = 0.83$ for infants.

For males (final row), the model performance is poor, though not as poor as for females. Males seem to be mistaken for females and infants at an equal rate, with an accuracy of $\frac{792}{792+351+385} = 0.52$ for males.

We can make an educated guess at physical interpretation of this output. Infants are likely much smaller than adults, resulting in accurate predictions on small individuals. The sexual dimorphism is likely limited, with males and females being similar, resulting in a high degree of confusion between them. Males are likely slightly smaller than females, because they are more often mistaken for infants.

Looking at the average values of our features, this provides support of this explanation:

In [12]:
data.groupby('Sex').mean()

,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings
Sex,,,,,,,,
F,0.579093,0.454732,0.158011,1.046532,0.446188,0.230689,0.302010,11.129304
I,0.427746,0.326494,0.107996,0.431363,0.191035,0.092010,0.128182,7.890462
M,0.561391,0.439287,0.151381,0.991459,0.432946,0.215545,0.281969,10.705497
